In [ ]:
import torch
print(torch.cuda.is_available())

print(torch.__version__)

In [ ]:
import cv2
import csv
import numpy as np
import dlib
from tqdm import tqdm

In [ ]:
import os

def load_images_and_labels(base_dir, emotions):
    images = []
    labels = []
    image_names = []

    for i, emotion in enumerate(emotions):
        emotion_dir = os.path.join(base_dir, emotion)
        for file_name in os.listdir(emotion_dir):
            file_path = os.path.join(emotion_dir, file_name)
            image = cv2.imread(file_path)
            if image is not None:
                modified_image_name = f"{os.path.splitext(file_name)[0]}.jpg"
                images.append(image)
                labels.append(emotion)  
                image_names.append(modified_image_name)  

    return images, labels, image_names


# emotions = ["Angry"]

# image, labels, image_name = load_images_and_labels('TIF_DF/Train/', emotions=emotions)

# image[1].shape

In [ ]:
def detect_landmarks(images, predictor_path="shape_predictor_68_face_landmarks.dat"):
    landmarks = []
    predictor = dlib.shape_predictor(predictor_path)

    for image in tqdm(images, desc="Detecting Landmarks"):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = gray.astype(np.uint8)
        height, width = gray.shape[:2]
        rect = dlib.rectangle(0, 0, width - 1, height - 1)
        landmark = predictor(gray, rect)
        landmark = np.array([[landmark.part(i).x, landmark.part(i).y] for i in range(68)])
        landmarks.append(landmark.tolist())
    return landmarks

landmarks = detect_landmarks(image)

In [ ]:
len(landmarks[1])

In [ ]:
def one_hot_encode(emotions, labels):
    emotion_dict = {emotion: idx for idx, emotion in enumerate(emotions)}
    one_hot = []

    for label in labels:
        one_hot_vector = [0.0] * len(emotions)
        if label in emotion_dict:
            one_hot_vector[emotion_dict[label]] = 1.0
        one_hot.append(one_hot_vector)

    return one_hot

In [ ]:
def save_to_csv(filename, image_names, landmarks, one_hot_labels, train_label):
    # Assuming image_names is a list, landmarks is a list of landmarks, and one_hot_labels is a one-hot encoded label
    data = {
        'Image Name': image_names,
        'Landmarks': landmarks,
        'Labels': one_hot_labels,
        'Actual_label': train_label
    }
    
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

In [ ]:
import os
import pandas as pd

def TIF_DF_Dataset():
    emotions = ["Angry", "Disgust", "Fear", "Happiness", "Neutral", "Sad", "Surprised"]

    train_images, train_labels, train_image_names = load_images_and_labels('TIF_DF/Train', emotions)
    test_images, test_labels, test_image_names = load_images_and_labels('TIF_DF/Test', emotions)

    print(train_labels)

    train_landmarks = detect_landmarks(train_images)
    test_landmarks = detect_landmarks(test_images)
    
    tifdf_train_one_hot = one_hot_encode(emotions, train_labels)
    tifdf_test_one_hot = one_hot_encode(emotions, test_labels)

    # Save to CSV with checks for file existence
    if not os.path.exists('tif_df_train.csv'):
        save_to_csv('tif_df_train.csv', train_image_names, train_landmarks, tifdf_train_one_hot, train_labels)
        print("tif_df_train.csv created successfully!")
    else:
        print("tif_df_train.csv already exists.")

    if not os.path.exists('tif_df_test.csv'):
        save_to_csv('tif_df_test.csv', test_image_names, test_landmarks, tifdf_test_one_hot, test_labels)
        print("tif_df_test.csv created successfully!")
    else:
        print("tif_df_test.csv already exists.")

In [ ]:
TIF_DF_Dataset()

In [ ]:
import os
import pandas as pd

def ChildNet_Dataset():
    emotions = ["Angry", "Disgust", "Fear", "Happiness", "Neutral", "Sad", "Surprised"]

    train_images, train_labels, train_image_names = load_images_and_labels('ChildNet/Train', emotions)
    test_images, test_labels, test_image_names = load_images_and_labels('ChildNet/Test', emotions)

    print(train_labels)

    train_landmarks = detect_landmarks(train_images)
    test_landmarks = detect_landmarks(test_images)
    
    tifdf_train_one_hot = one_hot_encode(emotions, train_labels)
    tifdf_test_one_hot = one_hot_encode(emotions, test_labels)

    # Save to CSV with checks for file existence
    if not os.path.exists('childnet_train.csv'):
        save_to_csv('childnet_train.csv', train_image_names, train_landmarks, tifdf_train_one_hot, train_labels)
        print("childnet_train.csv created successfully!")
    else:
        print("childnet_train.csv already exists.")

    if not os.path.exists('childnet_test.csv'):
        save_to_csv('childnet_test.csv', test_image_names, test_landmarks, tifdf_test_one_hot, test_labels)
        print("childnet_test.csv created successfully!")
    else:
        print("childnet_test.csv already exists.")

In [ ]:
df = pd.read_csv('tif_df_train.csv')
df.head(10)

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import cv2

df = pd.read_csv('tif_df_train.csv')

In [ ]:
# import os

# train_folder = 'TIF_DF/Train'
# emotions = ["Angry", "Disgust", "Fear", "Happiness", "Neutral", "Sad", "Surprised"]

# # List the files in the training folder under each emotion subfolder
# for emotion in emotions:
#     emotion_folder = os.path.join(train_folder, emotion)
#     if os.path.exists(emotion_folder) and os.path.isdir(emotion_folder):
#         print(f"Files in {emotion_folder}:")
#         for filename in os.listdir(emotion_folder):
#             file_path = os.path.join(emotion_folder, filename)
#             if os.path.isfile(file_path):
#                 print(f"  - {filename}")
#     else:
#         print(f"{emotion_folder} does not exist.")

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt

# Enable inline plotting for JupyterLab
# %matplotlib inline  # Uncomment this line if you're using JupyterLab

# Define paths
train_folder = 'TIF_DF/Train'  # Folder containing images
csv_file = 'tif_df_train.csv'  # Path to your CSV file with image names, labels, and landmarks

# Read CSV file
df = pd.read_csv(csv_file)

# Function to plot the image and overlay landmarks
def show_image_with_landmarks(image_name, landmarks, emotion_label):
    # Construct the correct image path based on emotion label
    emotion_folder = os.path.join(train_folder, emotion_label)
    image_path = os.path.join(emotion_folder, image_name)
    
    # Check if the image exists
    if not os.path.exists(image_path):
        print(f"Image {image_name} not found at {image_path}. Skipping.")
        return
    
    img = cv2.imread(image_path)
    
    # Check if image is read properly
    if img is None:
        print(f"Failed to read image {image_name}. Skipping.")
        return
    
    # Convert image from BGR (OpenCV default) to RGB (Matplotlib)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Plot the image
    plt.figure(figsize=(8, 8))
    plt.imshow(img)

    # Plot landmarks
    landmark_ls = eval(landmarks)  # Convert string representation of list to actual list
    for x, y in landmark_ls:
        plt.scatter(x, y, c='red', s=50, marker='x')  # Red color and 'x' marker
    
    plt.axis('off')  # Turn off axis
    plt.show()


# Loop through the rows in the CSV and plot images with landmarks
for index, row in df.iterrows():
    image_name = row['Image Name']  # Image name column in your CSV
    emotion_label = row['Actual_label']  # Emotion label in your CSV (Angry, Sad, etc.)
    
    # Check if the emotion label matches a folder in the train folder
    emotion_folder = os.path.join(train_folder, emotion_label)
    if not os.path.isdir(emotion_folder):
        print(f"Emotion folder {emotion_folder} does not exist. Skipping {emotion_label}.")
        continue
    
    # Check if the image exists in the emotion folder
    if image_name not in os.listdir(emotion_folder):
        print(f"Image {image_name} not found in {emotion_folder}. Skipping.")
        continue
    
    # Convert landmarks from string to a list of lists
    landmarks = row['Landmarks']  # Access the landmarks for the current row
    
    # Show the image and landmarks
    show_image_with_landmarks(image_name, landmarks, emotion_label)

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import ast 

%matplotlib inline  

# Define paths
train_folder = 'TIF_DF/Train' 
csv_file = 'tif_df_train.csv'  

# Read CSV file
df = pd.read_csv(csv_file)

def show_image_with_landmarks(image_name, landmarks, emotion_label):
    emotion_folder = os.path.join(train_folder, emotion_label)
    image_path = os.path.join(emotion_folder, image_name)
    
    if not os.path.exists(image_path):
        print(f"Image {image_name} not found at {image_path}. Skipping.")
        return
    
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"Failed to read image {image_name}. Skipping.")
        return
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(img)

    landmark_ls = ast.literal_eval(landmarks) 
    for x, y in landmark_ls:
        plt.scatter(x, y, c='red', s=50, marker='x')  
    
    plt.title(f"Image: {image_name}, Emotion: {emotion_label}")  
    plt.axis('off')  # Turn off axis
    plt.show()


for index, row in df.iterrows():
    # if index >= 10:  
    #     break
    
    image_name = row['Image Name'] 
    emotion_label = row['Actual_label']  
    
    emotion_folder = os.path.join(train_folder, emotion_label)
    if not os.path.isdir(emotion_folder):
        print(f"Emotion folder {emotion_folder} does not exist. Skipping {emotion_label}.")
        continue
    
    if image_name not in os.listdir(emotion_folder):
        print(f"Image {image_name} not found in {emotion_folder}. Skipping.")
        continue
    
    landmarks = row['Landmarks']  
    show_image_with_landmarks(image_name, landmarks, emotion_label)